<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Judy/American_Call_jax_Judy_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  13057      0 --:--:-- --:--:-- --:--:-- 13057
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 31 kB/s 
     |████████████████████████████████| 1.0 MB 31.4 MB/s 
+ set +ex
Installation succeeded!


# Test (Skip this if not trying to test, to make sure that functions are defined correctly in cells below without running this cell)

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

x_sub = jnp.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2]]).transpose()
y_sub = jnp.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(jax.numpy.linalg.lstsq(x_sub, y_sub)[0])
print(x_sub.dot(jax.numpy.linalg.lstsq(x_sub, y_sub)[0]))

[-1.0699916  2.9834177 -1.8135799]
[0.03674006 0.04589796 0.11752641 0.1519686  0.15641725]


In [ ]:
import numpy as np
x_sub = np.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2]]).transpose()
y_sub = np.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(np.linalg.lstsq(x_sub, y_sub, rcond=None)[0])
print(x_sub.dot(np.linalg.lstsq(x_sub, y_sub)[0]))

[-1.0699825   2.98339626 -1.81356745]
[0.03674038 0.04589812 0.11752626 0.15196848 0.15641716]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [ ]:
X = jnp.array([92.8,96.34,89.37,86.12])
X_2 = jnp.c_[np.ones_like(X), X, X**2]
Y = jnp.array([0,4.1759,6.7863,6.9345])

print(jax.numpy.linalg.lstsq(X_2, Y)[0])
print(X_2.dot(jax.numpy.linalg.lstsq(X_2, Y)[0]))

[ 0.01152701  0.52523154 -0.00521584]
[3.834999  2.2020226 5.2925377 6.5603523]


In [ ]:
X = np.array([92.8,96.34,89.37,86.12])
X_2 = np.c_[np.ones_like(X), X, X**2]
Y = np.array([0,4.1759,6.7863,6.9345])

print(np.linalg.lstsq(X_2, Y, rcond=None)[0])
print(X_2.dot(np.linalg.lstsq(X_2, Y, rcond=None)[0]))

[ 8.83369607e+02 -1.88603524e+01  1.00954440e-01]
[2.53238803 3.3613511  4.1427116  7.86024927]


In [ ]:
# call
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  #print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows
  #print(CF_matrix)

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-r[0]*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    print('CF_matrix')
    print(CF_matrix)
    print(jnp.where(X-K > ECV)[0])
    print(ITM_index[jnp.where(X-K > ECV)[0]])
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)
    print('CF_matrix after removing nonzeros')
    print(CF_matrix)

  return  CF_matrix.dot(jnp.exp(-r[0]*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out_avg
[[105.99862  100.0316   104.09863  111.01766  116.74928  109.565445]
 [ 90.274536  87.21661   96.076744  89.459885  91.10431   80.85022 ]
 [ 92.95862   96.04223   99.153305 104.86502  106.42198  104.403915]
 [ 93.90787   88.73891   91.71341   98.12489   96.75338   95.12549 ]
 [100.09762  112.7153   108.99028  119.56731  114.38698  103.237686]]
ITM_matrix
[[ True  True  True  True  True  True]
 [False False False False False False]
 [False False False  True  True  True]
 [False False False False False False]
 [ True  True  True  True  True  True]]
2
X
[116.74928 106.42198 114.38698]
Y
[9.549516  4.3965816 3.2322946]
ECV
[7.203638  3.633318  6.3367405]
CF_matrix
[[ 0.         0.         0.         0.        16.749283   9.565445 ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         6.421982   4.4039154]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.        1

In [ ]:
# call
# large numpaths & numsteps to test price here
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-r[0]*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-r[0]*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 1
numpaths = 1000

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 100
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([200.]*numstocks) # must be float

T = 2.0
K = 190.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r = drift, sigma_avg, T
# 100, 100, 0.01, 0.1443, 1
# option price should be around 6.3128
# test price: 6.3027

#S, K, r = drift, sigma, T
# 200, 190, 0.01, 0.1443, 2
# option price should be around 23.40856
# test price: 23.543825
####################################################################################

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

32.084904


In [ ]:
# call
# now change r to be an array
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
#r = drift # let r = drift to match B-S
r =jnp.array(np.random.random(numsteps) * 0.1)

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

8.339826
[0.19524175 0.25062665 0.22243044]


In [ ]:
# put
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  #print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows
  #print(CF_matrix)

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(K-out_avg[:,-1], 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(K-X > ECV).astype(int) * (K-X)) # X-K if X-K > ECV, else 0
    
    print('CF_matrix')
    print(CF_matrix)
    print(jnp.where(K-X > ECV)[0])
    print(ITM_index[jnp.where(K-X > ECV)[0]])
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(K-X > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)
    print('CF_matrix after removing nonzeros')
    print(CF_matrix)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out_avg
[[105.99862  100.0316   104.09863  111.01766  116.74928  109.565445]
 [ 90.274536  87.21661   96.076744  89.459885  91.10431   80.85022 ]
 [ 92.95862   96.04223   99.153305 104.86502  106.42198  104.403915]
 [ 93.90787   88.73891   91.71341   98.12489   96.75338   95.12549 ]
 [100.09762  112.7153   108.99028  119.56731  114.38698  103.237686]]
ITM_matrix
[[False False False False False False]
 [ True  True  True  True  True  True]
 [ True  True  True False False False]
 [ True  True  True  True  True  True]
 [False False False False False False]]
2
X
[91.10431 96.75338]
Y
[19.11789   4.866394]
ECV
[19.118011   4.8664856]
CF_matrix
[[ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.        19.14978  ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         4.8745117]
 [ 0.         0.         0.         0.         0.         0.       ]]
[]
[]
C

In [ ]:
# put
# large numpaths & numsteps to test price here
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(K-out_avg[:,-1], 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows

    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    ECV = X_matrix.dot(jnp.array(np.linalg.lstsq(X_matrix, Y, rcond=None)[0])) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(K-X > ECV).astype(int) * (K-X)) # K-X if K-X > ECV, else 0
     
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(K-X > ECV)[0]], -(i-1):], # only for paths where K-X > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 1
numpaths = 5000
numsteps = 50

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)* 0.25 * 0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.234

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5629
####################################################################################

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

9.562916


# Change to CF Array

In [ ]:
# change from CF matrix to CF array
# call
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  print('out')
  print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
  print('CF_array')
  print(CF_array)

  # update time T
  # ECV_array = jax.ops.index_update(x=ECV_array, 
  #                                  idx=jax.ops.index[:,-1], 
  #                                  y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    print(jnp.array(ITM_index[jnp.where(X-K > ECV)[0]]))
    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K if X-K > ECV, else 0
    print('CF_array')
    print(CF_array)

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 80.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out
[[[ 96.21578 ]
  [ 91.14775 ]
  [100.194565]
  [ 98.64239 ]
  [100.26974 ]
  [110.80411 ]]

 [[ 76.17836 ]
  [ 88.16234 ]
  [ 89.02843 ]
  [ 88.396324]
  [ 76.27098 ]
  [ 62.58006 ]]

 [[ 96.975716]
  [ 85.91724 ]
  [ 84.55148 ]
  [ 88.01227 ]
  [ 78.10432 ]
  [ 75.73002 ]]

 [[100.92803 ]
  [105.0573  ]
  [116.7805  ]
  [146.18767 ]
  [152.94846 ]
  [128.80319 ]]

 [[ 94.14551 ]
  [ 88.50854 ]
  [ 73.177025]
  [ 71.277466]
  [ 65.9051  ]
  [ 75.834854]]]
out_avg
[[ 96.21578   91.14775  100.194565  98.64239  100.26974  110.80411 ]
 [ 76.17836   88.16234   89.02843   88.396324  76.27098   62.58006 ]
 [ 96.975716  85.91724   84.55148   88.01227   78.10432   75.73002 ]
 [100.92803  105.0573   116.7805   146.18767  152.94846  128.80319 ]
 [ 94.14551   88.50854   73.177025  71.277466  65.9051    75.834854]]
CF_array
[30.804108  0.        0.       48.803192  0.      ]
ITM_matrix
[[ True  True  True  True  True  True]
 [False  True  True  True False False]
 [ True  True  True  True False 

In [ ]:
# change from CF matrix to CF array
# call
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 1000
numsteps = 100

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 90.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

14.875802
[0.75861514]


In [ ]:
##### Test(Judy)

In [1]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    #ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    ECV = X_matrix.dot(jnp.array(np.linalg.lstsq(X_matrix, Y, rcond=None)[0])) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
                                    y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5000
numsteps = 50

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.06]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.2*0.2
initial_stocks = jnp.array([40.]*numstocks) # must be float

T = 1.0
K = 40.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

2.3384314


In [3]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    #ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    ECV = X_matrix.dot(jnp.array(jnp.linalg.lstsq(X_matrix, Y, rcond=-1)[0])) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
                                    y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5000
numsteps = 50

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.06]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.2*0.2
initial_stocks = jnp.array([40.]*numstocks) # must be float

T = 1.0
K = 40.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

2.3400316
